In [5]:
import os
os.chdir('C:/Users/Zeddrex Navarra/Desktop/Career/Trial Codes/znavarra.github.io')

In [6]:
import os
current_dir = os.getcwd()
print(current_dir)

C:\Users\Zeddrex Navarra\Desktop\Career\Trial Codes\znavarra.github.io
